In [1]:
import TableParser
import pandas as pd
import tempfile, zipfile
import os
import numpy as np
import datetime
from collections import Counter
import time

In [14]:
class TestTableParser(TableParser.TableParserItem):
    def __init__(self):
        pass
    
    def parse(self, series: pd.Series):
        mistake = {
            "00:00:00":'','0:00':'',
            'Jan':'01','Feb': '02','Mar':'03','Apr':'04','May':'05','Jun':'06','Jul':'07','Aug':'08','Sept':'09','Oct':'10','Nov':'11','Dec':'12',
            'o': '0','l': '1','b': '6','g': '9','q': '9','t': '7','v': '5','G': '6','F': '7','Z': '2','Q': '2','B': '8','O': '0','D': '0','E': '3','A': '4','S': '5',
            r'[ -/]|[:-@]|[\[-_]|[{-~]': '-',
        }
        for key, val in mistake.items():
            series = series.str.strip().str.replace(key, val, regex=True)
        return (pd.to_datetime(series, errors="coerce"), [])

    def fix_xlsx(self, in_file):
        tmpfd, tmp = tempfile.mkstemp(dir=os.path.dirname(in_file))
        os.close(tmpfd)
        filename = '[Content_Types].xml'
        data = ''
        with zipfile.ZipFile(in_file, 'r') as zin:
            with zipfile.ZipFile(tmp, 'w') as zout:
                for item in zin.infolist():
                    if item.filename != filename:
                        zout.writestr(item, zin.read(item.filename))
                    else:
                        data = zin.read(filename).decode()
        os.remove(in_file)
        os.rename(tmp, in_file)
        data = data.replace('/xl/sharedStrings.xml', '/xl/SharedStrings.xml')
        with zipfile.ZipFile(in_file, mode='a', compression=zipfile.ZIP_DEFLATED) as zf:
            zf.writestr(filename, data)
            
    def orientation_determine(self,df):
        list_columns = []
        for j in range(df.shape[1]):
            list_columns.append(pd.to_numeric(df.iloc[:,j],errors='coerce').dropna())
        clear_df = pd.concat(list_columns,axis=1)
        if clear_df.shape[0] > clear_df.shape[1]:
            return True
        else: return False
        
    def find_start_point(self,clear_df):
        start_point = 0
        for i in range(clear_df.shape[0]):
            if len(clear_df.iloc[i,:].dropna()) > 2:
                start_point = i
                break
        return start_point
    
    def define_type_of_cell(self,clear_df,df_data,start_point):
        columns_types = {}
        for j in range(clear_df.shape[1]):
            baff = clear_df.iloc[:,j].dropna()
            if baff.sum() != 0:
                if baff.dtype == pd.Float64Dtype():
                    columns_types[j] = 'float'
                else: columns_types[j] = 'int'
            else:
                if type(testTableParser.parse(df_data.iloc[:,j].astype(str))[0][start_point]) == type(pd.to_datetime('31.01.2002')):
                    columns_types[j] = 'date'
                elif baff.dtype == pd.BooleanDtype():
                    columns_types[j] = 'bool'
                else: columns_types[j] = 'str'
        return columns_types
    
    def define_breakline(self,clear_df,columns_types):
        null_list,index_list= [],[]
        for j in range(clear_df.shape[1]):
            if columns_types[j] == 'int':
                null_list.append(clear_df.iloc[:,j])
        int_df = pd.concat(null_list,axis=1)
        count = Counter(columns_types.values())
        for i in range(int_df.shape[0]):
            if int_df.iloc[i,:].isnull().sum() == count.get('int'):
                index_list.append(i)

        return index_list

    def create_list_of_table(self,df,break_line,start_point):
        return [df.iloc[:break_line[0]+start_point,:],df.iloc[break_line[0]+start_point:,:]]
            

    def create_header(self,df,start_point,df_data):
        topic = df.iloc[:start_point,:].fillna(method='ffill',axis=1)
        list_topic = []
        for i in range(topic.shape[1]):
            list_topic.append("__".join(list(topic.iloc[:,i].astype(str))).replace("__nan","").replace("nan__",""))
        df_data.columns = list_topic
        return df_data   

    def to_numeric_func(self,df):
        list_columns = []
        for j in range(df.shape[1]):
            list_columns.append(pd.to_numeric(df.iloc[:,j],errors='coerce').convert_dtypes())
        clear_df = pd.concat(list_columns,axis=1)
#         start = time.time()
#         end = time.time()
#         print(end - start)
        return clear_df

    def json_creator(self,df,mistake,orientation,columns_types,break_line,index): 
        if len(break_line) == 0:
            return {"IsVerticalOrientation,":orientation,
               "Index rows":[df.index.tolist()],
               "columns types": columns_types,
                "mistake fix": mistake
                }
        else:
            return {"IsVerticalOrientation,":orientation,
               "Index rows":index,
               "columns types": columns_types,
                "mistake fix": mistake
                }
    
    def find_errorss(self,s,j):
        mistake = {'o': '0','l': '1','b': '6','g': '9','q': '9','t': '7','v': '5','f': '7','z': '2','e': '3','s': '5'}
        finded_error = []
        for key, val in mistake.items():
            if len(s[s.astype(str).str.contains(key).dropna()].index.to_list()) != 0:
                for i in range(len(s[s.astype(str).str.contains(key)].index.to_list())):
                    finded_error.append({   'row':s[s.astype(str).str.contains(key)].index.to_list()[i],
                                            'column':j,
                                            'new value':val,
                                            'old value':key,
                                            'probability':99.9,
                                            'user approve':False,
                                            'comment':False
                                        })
    #             finded_error.append([key,val,j,s[s.astype(str).str.contains(key)].index.to_list()])
        s = s.astype(str).str.strip().str.replace(key, val, regex=True)
        return [s,finded_error]

    def change_valuee(self,df,columns_types):
        # list_right_data = []
    #     df = pd.read_excel("file44.xlsx",header=None)
        mistake_index_list = []
        for j in range(df.shape[1]):
            if columns_types[j] == 'int' or columns_types[j] == 'float':
                buff = self.find_errorss(df.iloc[:,j],j)
                df.iloc[:,j] = pd.to_numeric(buff[0],errors='coerce').convert_dtypes().copy()
        #     list_right_data.append(pd.to_numeric(buff[0],errors='coerce').convert_dtypes())
                mistake_index_list.append(buff[1])
        return [df,np.concatenate(np.array(mistake_index_list), axis=None).tolist()]
    
    def all_action(self,df): # <-----    drive method
        orientation = self.orientation_determine(df)

        print(orientation)
        if orientation== False:
            df = df.T
            
        clear_df = self.to_numeric_func(df)
        start_point = self.find_start_point(clear_df)
        columns_types = self.define_type_of_cell(clear_df.iloc[start_point:,:],df.iloc[start_point:,:],start_point)
        break_line = self.define_breakline(clear_df.iloc[start_point:,:],columns_types)
        if  len(break_line) == 0:
            df_data_and_mistake = self.change_valuee(df.iloc[start_point:,:],columns_types)
            done_df = self.create_header(df,start_point,df_data_and_mistake[0])
#             print(df_data_and_mistake[1])
            json = self.json_creator(df,df_data_and_mistake[1],orientation,columns_types,break_line,[])
#             print(json)
            return [done_df,json]

        else:
            list_df = self.create_list_of_table(df,break_line,start_point)
            answer_df = []
            mistake = []
            index_list = []
            for i in range(len(list_df)):
                clear_df = self.to_numeric_func(list_df[i])
                start_point = self.find_start_point(clear_df)
                df_data_and_mistake = self.change_valuee(list_df[i].iloc[start_point:,:],columns_types)
                done_df = self.create_header(list_df[i],start_point,df_data_and_mistake[0])
                answer_df.append(done_df)
                mistake.append(df_data_and_mistake[1])
                index_list.append(list_df[i].index.tolist())
#             print(mistake)
            json = self.json_creator(df,mistake,orientation,columns_types,break_line,index_list)
#             print(json)
            return [answer_df,json]
            


        

In [15]:
# df = pd.read_csv("file44.csv",encoding='unicode_escape',header=None)
testTableParser = TestTableParser()
# df = pd.read_excel(open('file44.xlsx', 'rb'),header=None,sheet_name='Worksheet')
# df = pd.read_excel("file22.xlsx",header=None)
# print(df.shape)
# print(df.iloc[1,:])
# df = testTableParser.all_action(df)
# df[0]

xlsx = pd.ExcelFile('file44.xlsx')
names = xlsx.sheet_names
sheet_to_df_map = {}
for i  in range(len(names)):
    sheet_to_df_map[names[i]] = testTableParser.all_action(pd.read_excel(open('file44.xlsx', 'rb'),header=None,sheet_name=names[i]))
    

True
1       10107
2       10121
3       10134
4       10145
5       10159
        ...  
2819    10350
2820    10373
2821    10386
2822    10397
2823    10414
Name: 0, Length: 2823, dtype: object
1       30
2       34
3       41
4       45
5       49
        ..
2819    20
2820    29
2821    43
2822    34
2823    47
Name: 1, Length: 2823, dtype: object
1        95.7
2       81.35
3       94.74
4       83.26
5         100
        ...  
2819      100
2820      100
2821      100
2822    62.24
2823    65.52
Name: 2, Length: 2823, dtype: object
1        2
2        5
3        2
4        6
5       14
        ..
2819    15
2820     1
2821     4
2822     1
2823     9
Name: 3, Length: 2823, dtype: object
1          2871
2        2765.9
3       3884.34
4        3746.7
5       5205.27
         ...   
2819     2244.4
2820    3978.51
2821    5417.57
2822    2116.16
2823    3079.44
Name: 4, Length: 2823, dtype: object
1       1
2       2
3       3
4       3
5       4
       ..
2819    4
2820    1
2821

In [8]:
sheet_to_df_map.get('Worksheet')[1]

{'IsVerticalOrientation,': True,
 'Index rows': [[0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   82,
   83,
   84,
   85,
   86,
   87,
   88,
   89,
   90,
   91,
   92,
   93,
   94,
   95,
   96,
   97,
   98,
   99,
   100,
   101,
   102,
   103,
   104,
   105,
   106,
   107,
   108,
   109,
   110,
   111,
   112,
   113,
   114,
   115,
   116,
   117,
   118,
   119,
   120,
   121,
   122,
   123,
   124,
   125,
   126,
   127,
   128,
   129,
   130,
   131,
   132,

In [ ]:
# dict_keys(['Стадо 1 ', 'Стадо 2  ', 'Стадо 3 ', 'Стадо 4 ', 'Стадо 5 ', 'Стадо 6', 'Стадо 7', 'Стадо 8', 'Стадо 9', 'Стадо 10 ', 'Стадо 11', 'Стадо 12', 'Стадо 13'])

In [230]:
df = pd.read_excel("file44.xlsx",header=None)
df.iloc[5,19]

nan

In [336]:
# s = pd.Series(['a',234.0,32.9,'ayan',1,4,14,'12.4','34.2','21','a'])

# pd.to_numeric(s,errors='coerce').convert_dtypes()

In [337]:
# s[s.astype(str).str.contains('a')].index.to_list()

In [367]:
def find_errorss(s,j):
    mistake = {'o': '0','l': '1','b': '6','g': '9','q': '9','t': '7','v': '5','f': '7','z': '2','e': '3','a': '4','s': '5'}
    finded_error = []
    for key, val in mistake.items():
        if len(s[s.astype(str).str.contains(key).dropna()].index.to_list()) != 0:
            for i in range(len(s[s.astype(str).str.contains(key)].index.to_list())):
                finded_error.append({   'row':s[s.astype(str).str.contains(key)].index.to_list()[i],
                                        'column':j,
                                        'new value':val,
                                        'old value':key,
                                        'probability':99.9,
                                        'user approve':False,
                                        'comment':False
                                    })
#             finded_error.append([key,val,j,s[s.astype(str).str.contains(key)].index.to_list()])
        s = s.astype(str).str.strip().str.replace(key, val, regex=True)
    return [s,finded_error]

def change_valuee(df,columns_types):
    # list_right_data = []
#     df = pd.read_excel("file44.xlsx",header=None)
    mistake_index_list = []
    for j in range(df.shape[1]):
        if columns_types[j] == 'int' or columns_types[j] == 'float':
            buff = find_errorss(df.iloc[:,j],j)
            df.iloc[:,j] = pd.to_numeric(buff[0],errors='coerce').convert_dtypes().copy()
    #     list_right_data.append(pd.to_numeric(buff[0],errors='coerce').convert_dtypes())
            mistake_index_list.append(buff[1])
    return [df,np.concatenate(np.array(mistake_index_list), axis=None).tolist()]

In [369]:
df = pd.read_excel("file44.xlsx",header=None)
columns_types = {0: 'int', 1: 'int', 2: 'float', 3: 'int', 4: 'float', 5: 'date', 6: 'str', 7: 'int', 8: 'int', 9: 'int', 10: 'str', 11: 'int', 12: 'str', 13: 'str', 14: 'int', 15: 'str', 16: 'str', 17: 'str', 18: 'str', 19: 'int', 20: 'str', 21: 'str', 22: 'str', 23: 'str', 24: 'str'}
df_mistake = change_valuee(df.iloc[1:,:],columns_types)
df_mistake[1]

76

[{'row': 7,
  'column': 0,
  'new value': '1',
  'old value': 'l',
  'probability': 99.9,
  'user approve': False,
  'comment': False},
 {'row': 9,
  'column': 0,
  'new value': '1',
  'old value': 'l',
  'probability': 99.9,
  'user approve': False,
  'comment': False},
 {'row': 10,
  'column': 0,
  'new value': '9',
  'old value': 'q',
  'probability': 99.9,
  'user approve': False,
  'comment': False},
 {'row': 8,
  'column': 0,
  'new value': '0',
  'old value': 'D',
  'probability': 99.9,
  'user approve': False,
  'comment': False},
 {'row': 3,
  'column': 0,
  'new value': '3',
  'old value': 'E',
  'probability': 99.9,
  'user approve': False,
  'comment': False},
 {'row': 0,
  'column': 0,
  'new value': '4',
  'old value': 'A',
  'probability': 99.9,
  'user approve': False,
  'comment': False},
 {'row': 11,
  'column': 0,
  'new value': '5',
  'old value': 'S',
  'probability': 99.9,
  'user approve': False,
  'comment': False},
 {'row': 0,
  'column': 1,
  'new value': '0',

In [264]:
for key, val in mistake_index_list.items():
    for i in range(len(val)):
        print(val[i])

['l', '1', 0, [7, 9]]
['q', '9', 0, [10]]
['D', '0', 0, [8]]
['E', '3', 0, [3]]
['A', '4', 0, [0]]
['S', '5', 0, [11]]
['o', '0', 1, [0]]
['b', '6', 1, [3]]
['g', '9', 1, [8]]
['t', '7', 1, [10]]
['v', '5', 1, [7]]
['F', '7', 1, [9]]
[',', '.', 1, [0]]


[{'l': [7, 9], 'q': [10], 'D': [8], 'E': [3], 'A': [0], 'S': [11]},
 {'o': [0], 'b': [3], 'g': [8], 't': [10], 'v': [7], 'F': [9]}]

In [119]:
#         print([type(each) for each in df.iloc[:,4]])
# <class 'datetime.datetime'>

In [577]:
df.iloc[:,5].apply(type).value_counts()


<class 'str'>    2824
Name: 5, dtype: int64

In [118]:
 mistake = {'o': '0','l': '1','b': '6','g': '9','q': '9','t': '7','v': '5','G': '6','F': '7','Z': '2','Q': '2','B': '8','O': '0','D': '0','E': '3','A': '4','S': '5'}

In [121]:
mistake.get('l')

'1'

In [184]:
el = [3,4]
for i in range(len(el)):
    if el[i] == 3 or 4:
        print("hh")

hh
hh
